In [3]:
import torchaudio
import json
import os

In [6]:
with open('train.json') as f:

    train_json = json.load(f)

In [7]:
train_json

{'FVFB0_SX312.WAV': {'wav': '/rds/project/rds-xyBFuSj0hm0/MLMI2.M2022/data/TRAIN/DR1/FVFB0/SX312.WAV',
  'duration': 3.79525,
  'spk_id': 'FVFB0',
  'phn': 'sil dh ah z ae n sil s er z l sil b ih s sil t r ey f aa r w er sil d ih f y uw sil th ih ng sil dh ah m sil th r uw sil k eh r f ah l ih f er s sil t sil',
  'wrd': 'those answers will be straightforward if you think them through carefully first',
  'ground_truth_phn_ends': '2190 2650 3980 5260 7200 7760 8037 9420 10700 12160 13240 13840 14080 14840 16724 16968 17455 17838 18962 21730 23268 24271 25483 26840 27300 27450 28120 29585 29905 30363 31434 32009 33040 33600 34600 34990 35277 36551 37173 38410 38793 39560 40310 41410 42451 43568 45240 45751 46638 47640 49650 52409 54723 55210 56121 60640'},
 'FVFB0_SX222.WAV': {'wav': '/rds/project/rds-xyBFuSj0hm0/MLMI2.M2022/data/TRAIN/DR1/FVFB0/SX222.WAV.wav',
  'duration': 2.5344375,
  'spk_id': 'FVFB0',
  'phn': 'sil y aa w ih z sil k ih m ah sil w ah th sil p ae th sil l aa sil jh ih

# Features

## Use `dataloader` to get an utterance

In [6]:
from dataloader import get_dataloader
import torch
import numpy as np

In [7]:
loader = get_dataloader('train_fbank.json', 1, False)
fbank, lens, trans, dur = next(iter(loader))

In [8]:
print('The utterance has {} seconds.'.format(dur[0]))
print('Filter bank feature has {} frames and {} dimensions.'.format(
    fbank.shape[0], fbank.shape[2]))
print('The correspoinding transcription is: {}.'.format(trans[0]))

The utterance has 3.79525 seconds.
Filter bank feature has 378 frames and 23 dimensions.
The correspoinding transcription is: sil dh ah z ae n sil s er z l sil b ih s sil t r ey f aa r w er sil d ih f y uw sil th ih ng sil dh ah m sil th r uw sil k eh r f ah l ih f er s sil t sil.


## Plot FBANK

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

feature = np.transpose(fbank[:, 0, :].numpy())
plt.clf()
plt.figure(figsize=(10,2))
plt.imshow(feature, aspect='auto', origin='lower')
plt.colorbar()
plt.show()

# CTC model for ASR

## Obtain phoneme output units

In [10]:
# First find the unique phones in train.json, and then
# create a file named vocab.txt, each line in this 
# file is a unique phone, in total there should be 
# 40 lines

vocab = {}
phonemes = []
with open("vocab.txt") as f:
    for id, text in enumerate(f):
        vocab[text.strip()] = id
        phonemes.append(text)
phonemes = phonemes[1:]

In [11]:
# Count the number of times each phone appear in the training set
# and save the numbers correcponding to each phone in vocab.txt 
# into counts.txt

counts = []
with open("counts.txt") as f:
    for i in f:
        counts.append(int(i))

## Plot frequencies of graphemes

In [ ]:
plt.clf()
plt.figure(figsize=(10,2))
freq = np.array(counts) / np.sum(counts) * 100
plt.bar(phonemes, freq)
plt.show()

## Model & training configurations

In [13]:
from collections import namedtuple
if torch.cuda.is_available():
    device = "cuda:0"
else:
    device = "cpu"

args = {'seed': 123,
        'train_json': 'train_fbank.json',
        'val_json': 'dev_fbank.json',
        'test_json': 'test_fbank.json',
        'batch_size': 4,
        'num_layers': 1,
        'fbank_dims': 23,
        'model_dims': 128,
        'concat': 1,
        'lr': 0.5,
        'vocab': vocab,
        'report_interval': 50,
        'num_epochs': 20,
        'device': device,
       }

args = namedtuple('x', args)(**args)

In [ ]:
import models
model = models.BiLSTM(
    args.num_layers, args.fbank_dims * args.concat, args.model_dims, len(args.vocab))
num_params = sum(p.numel() for p in model.parameters())
print('Total number of model parameters is {}'.format(num_params))

## Training

In [ ]:
from datetime import datetime
from trainer import train
start = datetime.now()
model.to(args.device)
model_path = train(model, args)
end = datetime.now()
duration = (end - start).total_seconds()
print('Training finished in {} minutes.'.format(divmod(duration, 60)[0]))
print('Model saved to {}'.format(model_path))

## Decoding

In [16]:
### You can uncomment the following line and change model path to the model you want to decode
# model_path="checkpoints/20221110_120418/model_16"

In [ ]:
import torch
print('Loading model from {}'.format(model_path))
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

In [ ]:
from decoder import decode
results = decode(model, args, args.test_json)
print("SUB: {:.2f}%, DEL: {:.2f}%, INS: {:.2f}%, COR: {:.2f}%, PER: {:.2f}%".format(*results))